<a href="https://colab.research.google.com/github/aggrigorenko/sf_data_science/blob/main/Project_2_%D0%9D%D0%9D_%D0%B7%D0%B0%D0%BF%D1%80%D0%BE%D1%81%D1%8B_PostgreSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [96]:
import pandas as pd
import psycopg2

In [97]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [98]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [99]:
query_3_1 = f'''select count(*)
           from public.VACANCIES
           '''

In [100]:
df = pd.read_sql_query(query_3_1, connection)
df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [101]:
query_3_2 = f'''select count(*)
           from public.EMPLOYERS
           '''

In [102]:
df = pd.read_sql_query(query_3_2, connection)
df

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [103]:
query_3_3 = f'''select count(*)
           from public.AREAS
           '''

In [104]:
df = pd.read_sql_query(query_3_3, connection)
df

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [105]:
query_3_4 = f'''select count(*)
           from public.INDUSTRIES
           '''

In [106]:
df = pd.read_sql_query(query_3_4, connection)
df

,count
0,294


***

In [107]:
# Выводы по предварительному анализу данных.
# Посмотрели размеры таблиц, сколько всего вакансий, работодателей, 
# регионов и сфер деятельности.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [108]:
query_4_1 = f'''SELECT count(*) cnt,
    a.name area
FROM   
    public.VACANCIES v
JOIN public.AREAS a ON v.area_id = a.id   -- присоединение к табл. вакансий табл. регионов
GROUP BY a.name                           -- группировка по имени региона
ORDER BY 1 DESC                           -- сортировка по 1 столбцу в порядке убывания
limit 5                                   -- вывод первых 5 строк
'''

In [109]:
df = pd.read_sql_query(query_4_1, connection)
df

,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [110]:
query_4_2 = f'''SELECT count(*) cnt
    FROM public.VACANCIES 
where salary_from IS NOT NULL OR salary_to IS NOT NULL -- условие что заполнено хотя бы одно из двух полей с зарплатой
'''

In [111]:
df = pd.read_sql_query(query_4_2, connection)
df

,cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [112]:
query_4_3 = f'''SELECT 
    ROUND(AVG(salary_from)) slfr,  -- среднее нижней границы з/п
    ROUND(AVG(salary_to)) slto     -- среднее верхней границы з/п
    FROM public.VACANCIES 
'''

In [113]:
df = pd.read_sql_query(query_4_3, connection)
df

,slfr,slto
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [114]:
query_4_4 = f'''SELECT 
    schedule "Рабочий график",
    employment "Тип трудоустройства",
    count(*) cnt
    FROM public.VACANCIES
    group by 1, 2           -- группировка по 1 и 2 столбцам
    order by 3 desc         -- сортировка по 3 столбцу в порядке убывания
'''

In [115]:
df = pd.read_sql_query(query_4_4, connection)
df

,Рабочий график,Тип трудоустройства,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [116]:
query_4_5 = f'''SELECT 
    experience,
    count(*) cnt
    FROM public.VACANCIES
    group by 1             -- группировка по 1 столбцу
    order by 2             -- сортировка по 2 столбцу
'''

In [117]:
df = pd.read_sql_query(query_4_5, connection)
df

,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [118]:
# Выводы по детальному анализу вакансий.
# Выяснили распределение вакансий по регионам, у какого количества вакансий заполнены поля с з/п, средний размер з/п,
# определили как сочетаются рабочий график и тип трудоустройства - наиболее и наименее популярные сочетания,
# узнали какой опыт работы самый популярный.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [119]:
query_5_1 = f'''SELECT 
    e.name "Работодатель",
    count(*) cnt
FROM   
    public.VACANCIES v
LEFT JOIN public.employers e ON v.employer_id = e.id  -- присоединение к табл. вакансий табл. работодателей
GROUP BY e.name -- группировка по столбцу e.name
ORDER BY 2 DESC -- сортировка по столбцу "количество" в порядке убывания
limit 5'''

In [120]:
df = pd.read_sql_query(query_5_1, connection)
df

,Работодатель,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [121]:
query_5_2 = f'''select 
          a.name area_name,
          count(e.id) cnt_employers,
          count(v.id) cnt_vacancies
    
    from public.areas a 
         left join public.vacancies v on a.id = v.area_id -- присоединение к табл. регионов табл. вакансий
         left join public.employers e on a.id = e.area  -- присоединение табл. работодателей
    where v.id is null  -- условие отсутствия вакансии
    group by 1    -- группировка по регионам
    order by 2 desc  -- сортирока по кол-ву работодателей в порядке убывания
'''

In [122]:
df = pd.read_sql_query(query_5_2, connection)
df

,area_name,cnt_employers,cnt_vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
588,Кувандык,0,0
589,Куйбышев,0,0
590,Куйбышево (Ростовская область),0,0
591,Култаево,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [123]:
query_5_3 = f'''SELECT 
    e.name "Работодатель",
    count(DISTINCT v.area_id) "Колич. регионов"
FROM   
    public.employers e
    JOIN public.vacancies v ON v.employer_id = e.id -- присоединение к табл. работодателй табл. вакансий
    group by 1 -- группировка по 1 столбцу
    ORDER BY 2 DESC  -- сортировка по 2 столбцу в порядке убывания
'''

In [124]:
df = pd.read_sql_query(query_5_3, connection)
df

,Работодатель,Колич. регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [125]:
query_5_4 = f'''SELECT 
    count(*)
FROM   
    public.employers e
    LEFT JOIN EMPLOYERS_INDUSTRIES ei ON ei.employer_id = e.id -- присоединение к табл. работодателй табл. EMPLOYERS_INDUSTRIES
    where industry_id is null  -- условие неуказанной сферы деятельности
'''

In [126]:
df = pd.read_sql_query(query_5_4, connection)
df

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [127]:
query_5_5 = f'''SELECT 
    e.name "название компании",
    count(i.name) cnt
FROM   
    public.employers e
    LEFT JOIN EMPLOYERS_INDUSTRIES ei ON ei.employer_id = e.id   -- присоединение к табл. работодателй табл. EMPLOYERS_INDUSTRIES
    LEFT JOIN INDUSTRIES i ON ei.industry_id = i.id              -- присоединение табл. сфер деятельности работодателей
    group by 1 -- группировка по 1 столбцу
    having count(i.name) = 4             -- условие количества сфер деятельности = 4
    order by 1                           -- сортировка по 1 столбцу
    OFFSET 2 LIMIT 1                     -- вывод 3 строки
'''

In [128]:
df = pd.read_sql_query(query_5_5, connection)
df

,название компании,cnt
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [129]:
query_5_6 = f'''SELECT 
    count(e.name) cnt
FROM   
    public.employers e
    LEFT JOIN EMPLOYERS_INDUSTRIES ei ON ei.employer_id = e.id    -- присоединение к табл. работодателй табл. EMPLOYERS_INDUSTRIES
    LEFT JOIN INDUSTRIES i ON ei.industry_id = i.id               -- присоединение табл. сфер деятельности работодателей
    where i.name = 'Разработка программного обеспечения'          -- условие сферы деятельности = 'Разработка программного обеспечения'
'''

In [130]:
df = pd.read_sql_query(query_5_6, connection)
df

,cnt
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [131]:
# запрос таблицы городов-миллионников с сайта википедии
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' 
df_html = pd.read_html(url) # запрос таблиц
df_cities = df_html[1]  # выбор таблицы с городами-миллионниками
cities = df_cities['Город'].to_list() # преобразование столбца с городами в список
cities[13] = 'Воронеж'
cities = tuple(cities) # преобразование в кортеж
cities

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [132]:
query_5_7 = f'''select 
 a.name cities, 
 count(v.id) cnt 

from 
 public.vacancies v 
 left join public.areas a on a.id = v.area_id           -- присоединение к табл. вакансий табл. регионов
 left join public.employers e on v.employer_id = e.id   -- присоединение табл. работодателей

where e.name = 'Яндекс'   -- условие имя работодателя = 'Яндекс'
and a.name in {cities}    -- условие имя региона это город-миллионник
group by a.name           -- группировка по имени региона

union all                 -- объединение таблиц (добавление строки Total)

select 
 'Total', 
 count(distinct v.id) 

from 
 public.vacancies v 
 left join public.areas a on a.id = v.area_id 
 left join public.employers e on v.employer_id = e.id 
 
where e.name = 'Яндекс' 
and a.name in {cities} 
order by 2 desc            -- сортировка по 2 столбцу в порядке убывания
'''

In [133]:
df = pd.read_sql_query(query_5_7, connection)
df

,cities,cnt
0,Total,485
1,Москва,54
2,Санкт-Петербург,42
3,Екатеринбург,39
4,Нижний Новгород,36
5,Новосибирск,35
6,Воронеж,32
7,Краснодар,30
8,Самара,26
9,Уфа,26


***

In [134]:
# Выводы по анализу работодателей
# Определили самых популярных работодателей, их количественную распространённость по регионам,
# узнали каких и сколько сфер деятельности указали работодатели.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [137]:
query_6_1 = f'''select count(*)
FROM public.VACANCIES
     where LOWER(name) like '%data%' or LOWER(name) like '%данн%' -- условия для вакансий связанных с данными
'''

In [139]:
df = pd.read_sql_query(query_6_1, connection)
df

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [140]:
query_6_2 = f'''select count(*)
FROM public.VACANCIES
    WHERE  
    -- Условия для DS
    (LOWER(name) like '%data scientist%' OR 
    LOWER(name) like '%data science%' OR
    LOWER(name) like '%исследователь данных%' OR
    (name like '%ML%' AND LOWER(name) not like '%html%') OR 
    LOWER(name) like '%machine learning%' OR
    LOWER(name) like '%машинн%обучен%') AND
         -- Условия для Junior
    (LOWER(name) LIKE '%junior%' OR
    experience = 'Нет опыта' OR
    employment = 'Стажировка')
'''

In [141]:
df = pd.read_sql_query(query_6_2, connection)
df

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [142]:
query_6_3 = f'''select count(*)
FROM public.VACANCIES
    WHERE 
    -- Условия для DS
    (LOWER(name) like '%data scientist%' OR 
    LOWER(name) like '%data science%' OR
    LOWER(name) like '%исследователь данных%' OR
    (name like '%ML%' AND LOWER(name) not like '%html%') OR 
    LOWER(name) like '%machine learning%' OR
    LOWER(name) like '%машинн%обучен%') AND

    (key_skills like '%SQL%' OR      -- Условие для навыка SQL
    key_skills like '%postgres%' )   -- Условие для навыка postgres
'''

In [143]:
df = pd.read_sql_query(query_6_3, connection)
df

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [144]:
query_6_4 = f'''select count(*)
FROM public.VACANCIES
    WHERE 
    -- Условия для DS
    (LOWER(name) like '%data scientist%' OR 
    LOWER(name) like '%data science%' OR
    LOWER(name) like '%исследователь данных%' OR
    (name like '%ML%' AND LOWER(name) not like '%html%') OR 
    LOWER(name) like '%machine learning%' OR
    LOWER(name) like '%машинн%обучен%') AND

    LOWER(key_skills) like '%python%' -- Условие для навыка Python
'''

In [145]:
df = pd.read_sql_query(query_6_4, connection)
df

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [146]:
query_6_5 = f'''SELECT
    ROUND(AVG(1 + LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), ''))), 2) "Среднее количество навыков" -- формула расчёта среднего количества навыков
FROM public.VACANCIES
    WHERE 
    -- Условия для DS
    (LOWER(name) like '%data scientist%' OR 
    LOWER(name) like '%data science%' OR
    LOWER(name) like '%исследователь данных%' OR
    (name like '%ML%' AND LOWER(name) not like '%html%') OR 
    LOWER(name) like '%machine learning%' OR
    LOWER(name) like '%машинн%обучен%')
'''

In [147]:
df = pd.read_sql_query(query_6_5, connection)
df

,Среднее количество навыков
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [148]:
query_6_6 = f'''select
    experience "Опыт работы",
    ROUND(AVG(COALESCE((salary_to + salary_from)/2, salary_from, salary_to))) "Средняя з/п" -- формула расчёта средней з/п
    FROM public.vacancies

            --Условия DS
    WHERE (lower(name) LIKE '%data scientist%' 
        OR lower(name) LIKE '%data science%' 
        OR lower(name) LIKE '%исследователь данных%' 
        OR (name LIKE '%ML%' AND name NOT LIKE '%HTML%')
        OR lower(name) LIKE '%machine learning%'
        OR lower(name) LIKE '%машинн%обучен%')                                                      
    
    GROUP BY 1   -- группировка по столбцу "Опыт работы"
'''

In [149]:
df = pd.read_sql_query(query_6_6, connection)
df

,Опыт работы,Средняя з/п
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


***

In [150]:
# Выводы по предметному анализу
# Выяснили сколько вакансий имеет отношение к данным, сколько подходящих вакансий для начинающего дата-сайентиста; 
# определили сколько вакансий для DS, в которых в качестве ключевого навыка указан SQL, postgres и Python;
# узнали что в вакансиях для DS в среднем указывают более 6 ключевых навыков,
# вычислили среднюю з/п в вакансиях для DS и влияние опыта работы на её размер.

# Общий вывод по проекту

## 7. Дополнительные исследования данных

Средня з/п по всем вакансиям в зависимости от опыта работы.

In [151]:
query_7_1 = f'''select
            experience "Опыт работы",
            ROUND(AVG(COALESCE((salary_to + salary_from)/2, salary_from, salary_to))) "Средняя з/п" -- формула расчёта средней з/п
            FROM public.vacancies                                                   
            
            GROUP BY experience   --Группировка по столбцу experience
            ORDER BY 2            --Сортировка по столбцу средней з/п
        '''

In [152]:
df = pd.read_sql_query(query_7_1, connection)
df

,Опыт работы,Средняя з/п
0,Нет опыта,40472.0
1,От 1 года до 3 лет,76541.0
2,От 3 до 6 лет,133211.0
3,Более 6 лет,166256.0


In [153]:
# Общие выводы
# Количество вакансий на НН порядка 50.000, представлены 23.500 работодателями в 1362 регионах по 294 сферам деятельности.
# Средняя з/п от 71065 до 110537 руб.
# Наибольшее количество вакансий в крупных городах.
# Самые популярные тип рабочего графика и тип трудоустройства это "Полный день" и "Полная занятость".
# Количество вакансий и требуемый опыт работы не находятся в прямой зависимости, наибольшее число 
# вакансий у опыта "от 1 года до 3 лет", наименьшее у "более 6 лет", это скорее 
# всего связано с большими ожиданиями по з/п у работников с большим опытом.
# Компании из IT индустрии занимают лидирующее место на рынке труда по количеству вакансий, значительную долю
# имеют вакансии относящиеся к обработке данных. Средние з/п дата-сайентистов значительно выше среднерыночных.